<a href="https://colab.research.google.com/github/zavaleta/indices_climaticos/blob/master/NetCDF4_python2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NetCDF4

In [5]:
!pip install netcdf4

    100% |████████████████████████████████| 3.9MB 4.4MB/s 
    100% |████████████████████████████████| 286kB 25.6MB/s 


In [15]:
sampledata = 'http://dapds00.nci.org.au/thredds/dodsC/rr3/CMIP5/output1/CSIRO-BOM/ACCESS1-0/amip/mon/atmos/Amon/r1i1p1/latest/tas/tas_Amon_ACCESS1-0_amip_r1i1p1_197901-200812.nc'
sampledata

'http://dapds00.nci.org.au/thredds/dodsC/rr3/CMIP5/output1/CSIRO-BOM/ACCESS1-0/amip/mon/atmos/Amon/r1i1p1/latest/tas/tas_Amon_ACCESS1-0_amip_r1i1p1_197901-200812.nc'

## Xarray

Xarray works with file formats other than NetCDF as well, so some features like compression settings can be inconvenient to set.

In [2]:
import xarray

# Open a file
data = xarray.open_dataset(sampledata)

# Variables can be accessed either as properties or as a dict
surface_temperature = data.tas
surface_temperature = data['tas']

print("Variable:\n", surface_temperature)

# Same for attributes
units = surface_temperature.units
units = surface_temperature.attrs['units']

print()
print("Attribute:\n", units)

# Variables can be indexed numpy-style or pandas-style
d = surface_temperature[0, 0:10, 0:10]
d = surface_temperature.isel(time=0, lat=slice(0,10), lon=slice(0,10))
d = surface_temperature.sel(time='19790116T1200', lat=slice(-90,-80), lon=slice(0,20))

# Data can be saved to a new file easily
data.to_netcdf('data.nc')

ValueError: ignored

# NetCDF4

The netCDF4 library is a bare-bones library for working with NetCDF data. It doesn't have the bells and whistles of Xarray, but unlike Xarray it's a dedicated library, so features like compression and scale-and-offest are simpler to access.

In [7]:
import netCDF4

data = netCDF4.Dataset(sampledata)

# Variables can be accessed like a dict
surface_temperature = data['tas']
surface_temperature = data.variables['tas']

print("Variable:\n", surface_temperature)

# Attributes are accessed as properties of a variable
units = surface_temperature.units

print("Attribute:\n", units)

# Variables can be indexed numpy-style
data = surface_temperature[0, 0:10, 0:10]

# Data can't be copied to a new file easily

('Variable:\n', <type 'netCDF4._netCDF4.Variable'>
float32 tas(time, lat, lon)
    standard_name: air_temperature
    long_name: Near-Surface Air Temperature
    units: K
    cell_methods: time: mean
    cell_measures: area: areacella
    history: 2012-02-17T05:21:51Z altered by CMOR: Treated scalar dimension: 'height'. 2012-02-17T05:21:51Z altered by CMOR: replaced missing value flag (-1.07374e+09) with standard missing value (1e+20).
    coordinates: height
    missing_value: 1e+20
    _FillValue: 1e+20
    associated_files: baseURL: http://cmip-pcmdi.llnl.gov/CMIP5/dataLocation gridspecFile: gridspec_atmos_fx_ACCESS1-0_amip_r0i0p0.nc areacella: areacella_fx_ACCESS1-0_amip_r0i0p0.nc
unlimited dimensions: time
current shape = (360, 145, 192)
filling off
)
('Attribute:\n', u'K')


# Iris
https://scitools.org.uk/iris


While Xarray and netCDF4 both work similarly, the Iris library works a bit differently. Rather than accessing variables like a dictionary, Iris uses a list with a special function to get a variable by name. It also prefers using CF standard names, some special trickery is requried to get the variable by its name in the file.

Iris also keeps the file-level attributes with each of the variables - you can see below that it lists things like the title and metadata conventions

In [11]:
!pip install iris

In [12]:
import iris

data = iris.load(sampledata)

# Variables can be accessed like a list
surface_temperature = data[0]

# Iris prefers to use the standard_name to identify variables
surface_temperature = data.extract_strict('air_temperature')

# Getting variables by their own name can be done, but is complicated
surface_temperature = data.extract_strict(iris.Constraint(cube_func = lambda c: c.var_name == 'tas'))

print("Variable:\n", surface_temperature)

# Attributes can be accessed as properties
units = surface_temperature.units

print()
print("Attribute:\n", units)

# Variables can be indexed numpy-style or by special constraint objects
data = surface_temperature[0, 0:10, 0:10]
data = surface_temperature.extract(iris.Constraint(latitude=lambda x: 0 < x < 20))

# Data can be saved to a new file
iris.save(data, 'data.nc')

ERROR:root:An unexpected error occurred while tokenizing input
The following traceback may be corrupted or invalid
The error message is: ('EOF in multi-line string', (1, 0))



RuntimeError: ignored